In [1]:
from pymongo import MongoClient
# 네트워크 연결 규칙 : protocol://ip:port/path 
client = MongoClient('mongodb://192.168.0.91:27017/')

db_name = client["DB_TEST"]
col_target_name = db_name["COL_COMPARE_FEE"]


In [5]:
import pandas as pd
import numpy as np

'''
    일단 데이터 보기
    1. 각 증권사 에서 거래금액 별 결측치 있는지
    2. 구분 (변경 전후) 별 결측치 있는지 
    3. 계좌별 결측치 있는지 


'''

data_cursor = col_target_name.find({})
data_list = list(data_cursor)
print(len(data_list))
df_comp_fee = pd.DataFrame(data_list)
df_comp_fee.head()

590


,_id,회사명,수수료\n부과기준,기준일자,거래금액,구분,오프라인,HTS,ARS,스마트폰,오프라인.1,HTS.1,ARS.1,스마트폰.1,비고
0,67611223dde821a3a3d65747,비엔케이투자증권,/img/sub/ico_pdf.gif,2020/08/24,500만원,변경율,NaN,NaN,NaN,0.00000,NaN,NaN,NaN,0.0,NaN
1,67611223dde821a3a3d65748,부국증권,/img/sub/ico_hwp.gif,2011/11/18,10만원,변경율,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,67611223dde821a3a3d65749,미래에셋증권,/img/sub/ico_pdf.gif,2017/01/02,50만원,변경율,-0.01210,-0.71774,-0.28571,-0.28571,-0.01210,0.000,-0.92857,0.0,NaN
3,67611223dde821a3a3d6574a,한국투자증권,/img/sub/ico_pdf.gif,2021/11/25,10만원,변경율,NaN,NaN,0.00000,NaN,NaN,NaN,NaN,NaN,NaN
4,67611223dde821a3a3d6574b,유화증권,/img/sub/ico_pdf.gif,2012/05/02,50만원,변경율,0.10667,-0.02500,-0.00800,NaN,0.10667,-0.025,-0.00800,NaN,NaN


In [24]:
def convert_amount_to_float(amount_str):
    """거래금액 문자열을 숫자로 변환"""
    amount_str = amount_str.replace(' ', '')  # 공백 제거
    if '억원' in amount_str:
        # 1억원 -> 10000 (만원 단위)
        return float(amount_str.replace('억원', '')) * 10000
    else:
        # X만원 -> X
        return float(amount_str.replace('만원', ''))

In [27]:
import pandas as pd
import numpy as np

def fill_missing_smartphone(df, agg_func='mean'):
    """
    스마트폰.1 결측치를 채우는 함수
    
    Parameters:
    df : DataFrame
    agg_func : str, 집계 함수 ('mean', 'median', 'max' 등)
    """
    # 원본 데이터 복사
    df_filled = df.copy()
    
    # 변경후 데이터 중 스마트폰.1이 결측치인 레코드만 처리

    # 2. 같은 회사의 다른 거래금액 값의 비율 평균으로 채우기
    mask_still_missing = (df_filled['구분'] == '변경후') & (df_filled['스마트폰.1'].isna())
    for idx in df_filled[mask_still_missing].index:
        company = df_filled.loc[idx, '회사명']
        current_amount = convert_amount_to_float(df_filled.loc[idx, '거래금액'])
        
        # 같은 회사의 다른 변경후 데이터 추출
        other_records = df_filled[(df_filled['회사명'] == company) & 
                                (df_filled['구분'] == '변경후') & 
                                (df_filled['스마트폰.1'].notna())]
        
        if len(other_records) > 0:
            # 각 거래금액 대비 스마트폰.1 비율 계산
            ratios = []
            amounts = []  # 출력용 거래금액 저장
            for _, row in other_records.iterrows():
                amount = convert_amount_to_float(row['거래금액'])
                ratio = row['스마트폰.1'] / amount
                ratios.append(ratio)
                amounts.append(f"{row['거래금액']}:{ratio:.6f}")
            
            # 비율의 평균 계산
            avg_ratio = sum(ratios) / len(ratios)
            # 현재 거래금액에 평균 비율 적용
            fill_value = current_amount * avg_ratio
            
            print(f"case 2 : {idx}, {df_filled.loc[idx, ['회사명', '거래금액', '구분', '스마트폰', '스마트폰.1']]}")
            # print(f"        거래금액별 비율: {amounts}")
            # print(f"        평균 비율: {avg_ratio:.6f}")
            # print(f"        적용값 ({df_filled.loc[idx, '거래금액']} × {avg_ratio:.6f}): {fill_value:.6f}")
            
            df_filled.loc[idx, '스마트폰.1'] = fill_value
    
    # 3. 같은 레코드의 스마트폰 값으로 채우기
    mask_still_missing = (df_filled['구분'] == '변경후') & (df_filled['스마트폰.1'].isna())
    for idx in df_filled[mask_still_missing].index:
        if not pd.isna(df_filled.loc[idx, '스마트폰']):
            print(f"case 3 : {idx}, {df_filled.loc[idx, ['회사명', '거래금액', '구분', '스마트폰', '스마트폰.1']]}")
            df_filled.loc[idx, '스마트폰.1'] = df_filled.loc[idx, '스마트폰']

    mask_target = (df_filled['구분'] == '변경후') & (df_filled['스마트폰.1'].isna())
    # 1. 같은 회사, 같은 거래금액의 변경전 값과 변경율로 채우기
    for idx in df_filled[mask_target].index:
        company = df_filled.loc[idx, '회사명']
        amount = df_filled.loc[idx, '거래금액']
        
        # 같은 회사, 같은 거래금액의 변경전 값과 변경율 찾기
        prev_value = df_filled[(df_filled['회사명'] == company) & 
                             (df_filled['거래금액'] == amount) & 
                             (df_filled['구분'] == '변경전')]['스마트폰.1'].values
        
        rate = df_filled[(df_filled['회사명'] == company) & 
                        (df_filled['거래금액'] == amount) & 
                        (df_filled['구분'] == '변경율')]['스마트폰.1'].values
        
        # if len(prev_value) > 0 and len(rate) > 0 and not np.isnan(prev_value[0]) and not np.isnan(rate[0]):
        if len(prev_value) > 0 and not np.isnan(prev_value[0]) :
            print(f"case 1 : {idx}, {df_filled.loc[idx, ['회사명', '거래금액', '구분', '스마트폰', '스마트폰.1']]}")
            df_filled.loc[idx, '스마트폰.1'] = prev_value[0] * (1 + rate[0])

    # 남은 결측치 확인
    remaining_missing = df_filled[(df_filled['구분'] == '변경후') & 
                                (df_filled['스마트폰.1'].isna())]
    
    return df_filled, remaining_missing

# 함수 사용
df_filled, remaining_missing = fill_missing_smartphone(df_comp_fee, agg_func='mean')

# 결과 출력
print("=== 채우기 전 변경후 결측치 수 ===")
print(len(df_comp_fee[(df_comp_fee['구분'] == '변경후') & (df_comp_fee['스마트폰.1'].isna())]))

print("\n=== 채운 후 변경후 결측치 수 ===")
print(len(remaining_missing))

print("\n=== 남은 결측치가 있는 레코드 ===")
if len(remaining_missing) > 0:
    print(remaining_missing[['회사명', '거래금액', '구분', '스마트폰', '스마트폰.1']].sort_values(['회사명', '거래금액', '구분']))
else:
    print("남은 결측치가 없습니다.")

case 2 : 424, 회사명       현대차증권
거래금액      100만원
구분          변경후
스마트폰        NaN
스마트폰.1      NaN
Name: 424, dtype: object
case 2 : 475, 회사명          현대차증권
거래금액           1억원
구분             변경후
스마트폰      198100.0
스마트폰.1         NaN
Name: 475, dtype: object
case 2 : 484, 회사명       하나증권
거래금액       1억원
구분         변경후
스마트폰       NaN
스마트폰.1     NaN
Name: 484, dtype: object
case 2 : 522, 회사명        현대차증권
거래금액      1000만원
구분           변경후
스마트폰         NaN
스마트폰.1       NaN
Name: 522, dtype: object
case 3 : 379, 회사명        토스증권
거래금액      500만원
구분          변경후
스마트폰      750.0
스마트폰.1      NaN
Name: 379, dtype: object
case 3 : 382, 회사명       토스증권
거래금액      50만원
구분         변경후
스마트폰      75.0
스마트폰.1     NaN
Name: 382, dtype: object
case 3 : 390, 회사명       카카오페이증권
거래금액        100만원
구분            변경후
스마트폰        150.0
스마트폰.1        NaN
Name: 390, dtype: object
case 3 : 396, 회사명       카카오페이증권
거래금액        500만원
구분            변경후
스마트폰        750.0
스마트폰.1        NaN
Name: 396, dtype: object
case 3 : 403, 회사명 

In [14]:
import pandas as pd
import numpy as np

# 회사별, 구분별, 거래금액별로 스마트폰과 스마트폰.1 결측치 각각 확인
missing_by_company = df_comp_fee.groupby(['회사명', '구분', '거래금액']).apply(
    lambda x: pd.Series({
        '스마트폰_결측치': x['스마트폰'].isnull().sum(),
        '스마트폰.1_결측치': x['스마트폰.1'].isnull().sum()
    })
).reset_index()

# 구분별로 나누어 결과 확인
for category in ['변경전', '변경후', '변경율']:
    print(f"\n=== {category} 구분의 결측치 현황 ===")
    category_results = missing_by_company[missing_by_company['구분'] == category]
    
    # 결측치가 있는 경우만 표시
    category_missing = category_results[
        (category_results['스마트폰_결측치'] > 0) | 
        (category_results['스마트폰.1_결측치'] > 0)
    ]
    
    if not category_missing.empty:
        print(f"\n{category} 구분에서 결측치가 있는 경우:")
        print(category_missing.sort_values(['회사명', '거래금액']))
    else:
        print(f"\n{category} 구분에서 결측치가 없습니다.")

    # 전체 현황도 보고 싶다면:
    print(f"\n{category} 구분의 전체 현황:")
    print(category_results.sort_values(['회사명', '거래금액']))


=== 변경전 구분의 결측치 현황 ===

변경전 구분에서 결측치가 있는 경우:
        회사명   구분    거래금액  스마트폰_결측치  스마트폰.1_결측치
24   DS투자증권  변경전  1000만원         1           1
25   DS투자증권  변경전   100만원         1           1
26   DS투자증권  변경전    10만원         1           1
27   DS투자증권  변경전     1억원         1           1
28   DS투자증권  변경전   500만원         1           1
..      ...  ...     ...       ...         ...
579    흥국증권  변경전   100만원         1           1
580    흥국증권  변경전    10만원         1           1
581    흥국증권  변경전     1억원         1           1
582    흥국증권  변경전   500만원         1           1
583    흥국증권  변경전    50만원         1           1

[73 rows x 5 columns]

변경전 구분의 전체 현황:
        회사명   구분    거래금액  스마트폰_결측치  스마트폰.1_결측치
6    DB금융투자  변경전  1000만원         0           0
7    DB금융투자  변경전   100만원         0           0
8    DB금융투자  변경전    10만원         0           0
9    DB금융투자  변경전     1억원         0           0
10   DB금융투자  변경전   500만원         0           0
..      ...  ...     ...       ...         ...
579    흥국증권  변경전   100

/tmp/ipykernel_14629/1667446693.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_by_company = df_comp_fee.groupby(['회사명', '구분', '거래금액']).apply(


In [ ]:
# 증권사별, 거래금액별로 스마트폰 컬럼의 결측치 확인
missing_by_company = df_comp_fee.groupby(['회사명', '거래금액']).apply(
    lambda x: pd.Series({
        '스마트폰_결측치': x['스마트폰'].isnull().sum(),
        '스마트폰.1_결측치': x['스마트폰.1'].isnull().sum()
    })
).reset_index()

print("증권사별 거래금액에 따른 스마트폰 결측치:")
print(missing_by_company)

# 결측치가 있는 경우만 보기 원한다면:
missing_only = missing_by_company[
    (missing_by_company['스마트폰_결측치'] > 0) | 
    (missing_by_company['스마트폰.1_결측치'] > 0)
]
print("\n결측치가 있는 경우만 표시:")
print(missing_only)

증권사별 거래금액에 따른 스마트폰 결측치:
        회사명    거래금액  스마트폰_결측치  스마트폰.1_결측치
0    DB금융투자  1000만원         0           0
1    DB금융투자   100만원         0           0
2    DB금융투자    10만원         0           0
3    DB금융투자     1억원         0           0
4    DB금융투자   500만원         0           0
..      ...     ...       ...         ...
211    흥국증권   100만원         3           3
212    흥국증권    10만원         3           3
213    흥국증권     1억원         3           3
214    흥국증권   500만원         3           3
215    흥국증권    50만원         3           3

[216 rows x 4 columns]

결측치가 있는 경우만 표시:
        회사명    거래금액  스마트폰_결측치  스마트폰.1_결측치
6    DS투자증권  1000만원         3           3
7    DS투자증권   100만원         3           3
8    DS투자증권    10만원         3           3
9    DS투자증권     1억원         3           3
10   DS투자증권   500만원         3           3
..      ...     ...       ...         ...
211    흥국증권   100만원         3           3
212    흥국증권    10만원         3           3
213    흥국증권     1억원         3           3
214    흥국증권

/tmp/ipykernel_14629/1684474310.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_by_company = df_comp_fee.groupby(['회사명', '거래금액']).apply(


In [13]:

# 1. 먼저 각 증권사별, 거래금액별 결측치 현황 확인
missing_by_company = df_comp_fee.groupby(['회사명', '거래금액']).apply(
    lambda x: pd.Series({
        '스마트폰_결측치': x['스마트폰'].isnull().sum(),
        '스마트폰.1_결측치': x['스마트폰.1'].isnull().sum()
    })
).reset_index()

# 2. 증권사별로 전체 거래금액 수와 결측치가 있는 거래금액 수 비교
company_summary = df_comp_fee.groupby('회사명').agg({
    '거래금액': 'count',  # 전체 거래금액 수
}).rename(columns={'거래금액': '전체_거래금액_수'})

missing_summary = missing_by_company[
    (missing_by_company['스마트폰_결측치'] > 0) | 
    (missing_by_company['스마트폰.1_결측치'] > 0)
].groupby('회사명').size().rename('결측치_있는_거래금액_수')

# 3. 결과 합치기
result = pd.merge(company_summary, missing_summary, on='회사명', how='left')
result['결측치_있는_거래금액_수'] = result['결측치_있는_거래금액_수'].fillna(0)

# 4. 부분적으로 결측치가 있는 증권사 찾기 (전체 거래금액 수와 결측치 있는 거래금액 수가 다른 경우)
partial_missing = result[
    (result['결측치_있는_거래금액_수'] > 0) & 
    (result['결측치_있는_거래금액_수'] < result['전체_거래금액_수'])
]

print("부분적으로 결측치가 있는 증권사:")
print(partial_missing)

# 5. 해당 증권사들의 상세 현황 보기
if not partial_missing.empty:
    partial_missing_details = missing_by_company[
        missing_by_company['회사명'].isin(partial_missing.index)
    ].sort_values(['회사명', '거래금액'])
    
    print("\n해당 증권사들의 거래금액별 결측치 현황:")
    print(partial_missing_details)

부분적으로 결측치가 있는 증권사:
           전체_거래금액_수  결측치_있는_거래금액_수
회사명                                
DS투자증권            18            6.0
KB증권              18            6.0
리딩투자증권            16            6.0
부국증권              10            1.0
비엔피파리바증권          18            6.0
삼성증권              18            6.0
신한투자증권            18            6.0
유진투자증권            18            6.0
유화증권              18            6.0
카카오페이증권           18            6.0
코리아에셋투자증권         18            6.0
하나증권              18            1.0
한국투자증권            18            6.0
현대차증권             14            4.0
흥국증권              18            6.0

해당 증권사들의 거래금액별 결측치 현황:
        회사명    거래금액  스마트폰_결측치  스마트폰.1_결측치
6    DS투자증권  1000만원         3           3
7    DS투자증권   100만원         3           3
8    DS투자증권    10만원         3           3
9    DS투자증권     1억원         3           3
10   DS투자증권   500만원         3           3
..      ...     ...       ...         ...
211    흥국증권   100만원         3           3
212    흥국

/tmp/ipykernel_14629/2472698627.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_by_company = df_comp_fee.groupby(['회사명', '거래금액']).apply(


In [12]:
# 증권사별, 거래금액별로 스마트폰 컬럼의 결측치 확인
missing_by_company = df_comp_fee.groupby(['회사명', '구분']).apply(
    lambda x: pd.Series({
        '스마트폰_결측치': x['스마트폰'].isnull().sum(),
        '스마트폰.1_결측치': x['스마트폰.1'].isnull().sum()
    })
).reset_index()

print("증권사별 거래금액에 따른 스마트폰 결측치:")
print(missing_by_company)

# 결측치가 있는 경우만 보기 원한다면:
missing_only = missing_by_company[
    (missing_by_company['스마트폰_결측치'] > 0) | 
    (missing_by_company['스마트폰.1_결측치'] > 0)
]
print("\n결측치가 있는 경우만 표시:")
print(missing_only)

증권사별 거래금액에 따른 스마트폰 결측치:
        회사명   구분  스마트폰_결측치  스마트폰.1_결측치
0    DB금융투자  변경율         0           0
1    DB금융투자  변경전         0           0
2    DB금융투자  변경후         0           0
3    DS투자증권  변경율         6           6
4    DS투자증권  변경전         6           6
..      ...  ...       ...         ...
99    현대차증권  변경전         1           1
100   현대차증권  변경후         2           3
101    흥국증권  변경율         6           6
102    흥국증권  변경전         6           6
103    흥국증권  변경후         6           6

[104 rows x 4 columns]

결측치가 있는 경우만 표시:
           회사명   구분  스마트폰_결측치  스마트폰.1_결측치
3       DS투자증권  변경율         6           6
4       DS투자증권  변경전         6           6
5       DS투자증권  변경후         6           6
9         KB증권  변경율         6           6
10        KB증권  변경전         6           6
27       도이치증권  변경후         6           6
28      리딩투자증권  변경율         5           5
29      리딩투자증권  변경전         5           5
30      리딩투자증권  변경후         6           6
37        부국증권  변경율         1           1
38   

/tmp/ipykernel_14629/127289994.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_by_company = df_comp_fee.groupby(['회사명', '구분']).apply(


In [ ]:
# 결과 출력
print("1. 증권사별 거래금액 결측치:")
print(missing_by_amount)
print("\n2. 구분별 결측치:")
print(missing_by_change)
print("\n3. 계좌 채널별 결측치:")
print(missing_by_account)